In [10]:
import panel as pn
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
db_config = {
    'user': 'postgres',
    'password': '1412',
    'host': 'localhost',
    'port': '5432',
    'database': 'Gerenciamento_Remedios'
}


engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")
con = engine.connect()



In [11]:

def read_clientes(id_cliente=None, nome_cliente=None):
    try:
        if id_cliente is not None:
            query = text("SELECT * FROM farmacia.cliente WHERE id_cliente = :id_cliente")
            df = pd.read_sql(query, con, params={"id_cliente": id_cliente})
        elif nome_cliente is not None:
            query = text("SELECT * FROM farmacia.cliente WHERE nome_cliente ILIKE :nome_cliente")
            df = pd.read_sql(query, con, params={"nome_cliente": f"%{nome_cliente}%"})
        else:
            query = "SELECT * FROM farmacia.cliente"
            df = pd.read_sql(query, con)
        return df
    except SQLAlchemyError as e:
        return str(e)


def create_cliente(nome_cliente, cpf, endereco_rua, endereco_numero, endereco_bairro, endereco_cidade):
    try:
        query = text("""
            INSERT INTO farmacia.cliente 
            (nome_cliente, cpf, endereco_rua, endereco_numero, endereco_bairro, endereco_cidade)
            VALUES (:nome_cliente, :cpf, :endereco_rua, :endereco_numero, :endereco_bairro, :endereco_cidade)
        """)
        con.execute(query, {
            "nome_cliente": nome_cliente,
            "cpf": cpf,
            "endereco_rua": endereco_rua,
            "endereco_numero": endereco_numero,
            "endereco_bairro": endereco_bairro,
            "endereco_cidade": endereco_cidade
        })
        con.commit()
        return "Cliente adicionado com sucesso!"
    except SQLAlchemyError as e:
        return str(e)


def update_cliente(id_cliente, nome_cliente, cpf, endereco_rua, endereco_numero, endereco_bairro, endereco_cidade):
    try:
        query = text("""
            UPDATE farmacia.cliente 
            SET nome_cliente = :nome_cliente, cpf = :cpf, endereco_rua = :endereco_rua, 
                endereco_numero = :endereco_numero, endereco_bairro = :endereco_bairro, 
                endereco_cidade = :endereco_cidade 
            WHERE id_cliente = :id_cliente
        """)
        con.execute(query, {
            "id_cliente": id_cliente,
            "nome_cliente": nome_cliente,
            "cpf": cpf,
            "endereco_rua": endereco_rua,
            "endereco_numero": endereco_numero,
            "endereco_bairro": endereco_bairro,
            "endereco_cidade": endereco_cidade
        })
        con.commit()
        return "Cliente atualizado com sucesso!"
    except SQLAlchemyError as e:
        return str(e)

def delete_cliente(id_cliente):
    try:
        query = text("DELETE FROM farmacia.cliente WHERE id_cliente = :id_cliente")
        con.execute(query, {"id_cliente": id_cliente})
        con.commit()
        return "Cliente excluído com sucesso!"
    except SQLAlchemyError as e:
        return str(e)


In [12]:

def update_table(id_cliente=None, nome_cliente=None):
    df = read_clientes(id_cliente, nome_cliente)
    table.value = df

def on_filter_button_click(event):
    id_filter = id_filter_input.value if id_filter_input.value else None
    nome_filter = nome_filter_input.value if nome_filter_input.value else None
    update_table(id_cliente=id_filter, nome_cliente=nome_filter)
    output_pane.object = "Tabela atualizada com base no filtro."


In [13]:

id_filter_input = pn.widgets.IntInput(name='Filtrar por ID', width=150)
nome_filter_input = pn.widgets.TextInput(name='Filtrar por Nome', placeholder='Nome do Cliente', width=300)
filter_button = pn.widgets.Button(name='Aplicar Filtro', button_type='primary')

filter_button.on_click(on_filter_button_click)


Watcher(inst=Button(button_type='primary', name='Aplicar Filtro'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_filter_button_click at 0x00000216974E11C0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [14]:

nome_input = pn.widgets.TextInput(name='Nome', placeholder='Nome do Cliente', width=300)
cpf_input = pn.widgets.TextInput(name='CPF', placeholder='CPF do Cliente', width=150)
endereco_rua_input = pn.widgets.TextInput(name='Endereço - Rua', placeholder='Rua', width=300)
endereco_numero_input = pn.widgets.IntInput(name='Endereço - Número', width=150)
endereco_bairro_input = pn.widgets.TextInput(name='Endereço - Bairro', placeholder='Bairro', width=300)
endereco_cidade_input = pn.widgets.TextInput(name='Endereço - Cidade', placeholder='Cidade', width=300)
create_button = pn.widgets.Button(name='Adicionar Cliente', button_type='primary')

 
def on_create_button_click(event):
    message = create_cliente(
        nome_input.value, cpf_input.value, endereco_rua_input.value, 
        endereco_numero_input.value, endereco_bairro_input.value, endereco_cidade_input.value
    )
    update_table()
    output_pane.object = message

create_button.on_click(on_create_button_click)


Watcher(inst=Button(button_type='primary', name='Adicionar Cliente'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_create_button_click at 0x00000216B2B177E0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [15]:

id_input = pn.widgets.IntInput(name='ID do Cliente', width=150)
update_nome_input = pn.widgets.TextInput(name='Novo Nome', placeholder='Novo Nome do Cliente', width=300)
update_cpf_input = pn.widgets.TextInput(name='Novo CPF', placeholder='Novo CPF', width=150)
update_endereco_rua_input = pn.widgets.TextInput(name='Nova Rua', placeholder='Nova Rua', width=300)
update_endereco_numero_input = pn.widgets.IntInput(name='Novo Número', width=150)
update_endereco_bairro_input = pn.widgets.TextInput(name='Novo Bairro', placeholder='Novo Bairro', width=300)
update_endereco_cidade_input = pn.widgets.TextInput(name='Nova Cidade', placeholder='Nova Cidade', width=300)
update_button = pn.widgets.Button(name='Atualizar Cliente', button_type='warning')

def on_update_button_click(event):
    message = update_cliente(
        id_input.value, update_nome_input.value, update_cpf_input.value, 
        update_endereco_rua_input.value, update_endereco_numero_input.value, 
        update_endereco_bairro_input.value, update_endereco_cidade_input.value
    )
    update_table()
    output_pane.object = message

update_button.on_click(on_update_button_click)


Watcher(inst=Button(button_type='warning', name='Atualizar Cliente'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_update_button_click at 0x00000216B22F5940>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [16]:

delete_id_input = pn.widgets.IntInput(name='ID do Cliente para Excluir', width=150)
delete_button = pn.widgets.Button(name='Excluir Cliente', button_type='danger')

 
def on_delete_button_click(event):
    message = delete_cliente(delete_id_input.value)
    update_table()
    output_pane.object = message

delete_button.on_click(on_delete_button_click)


Watcher(inst=Button(button_type='danger', name='Excluir Cliente'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_delete_button_click at 0x00000216B22F65C0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [17]:

table = pn.widgets.DataFrame(read_clientes(), name='Tabela de Clientes', width=600)
output_pane = pn.pane.Markdown("Mensagens aparecerão aqui", width=600)


In [18]:

layout = pn.Column(
    pn.pane.Markdown("# Gerenciamento de Clientes", sizing_mode='stretch_width'),
    pn.Row(
        pn.Column(id_filter_input, nome_filter_input, filter_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(nome_input, cpf_input, endereco_rua_input, endereco_numero_input, endereco_bairro_input, endereco_cidade_input, create_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(id_input, update_nome_input, update_cpf_input, update_endereco_rua_input, update_endereco_numero_input, update_endereco_bairro_input, update_endereco_cidade_input, update_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(delete_id_input, delete_button),
        sizing_mode='stretch_width'
    ),
    table,
    output_pane,
    sizing_mode='stretch_width'
)

layout.show()


Launching server at http://localhost:60040
